In [1]:
!pip install Keras 2.12.0
!pip install numpy 1.24.3
!pip install opencv-python 4.10.0.84
!pip install tensorflow 2.12.1
!pip install Matplotlib 3.9.4
!pip install imutils 0.5.3
!pip install Keras 2.12.0

ERROR: Could not find a version that satisfies the requirement 2.12.0 (from versions: none)
ERROR: No matching distribution found for 2.12.0


ERROR: Could not find a version that satisfies the requirement 1.24.3 (from versions: none)
ERROR: No matching distribution found for 1.24.3


ERROR: Could not find a version that satisfies the requirement 4.10.0.84 (from versions: none)
ERROR: No matching distribution found for 4.10.0.84


ERROR: Could not find a version that satisfies the requirement 2.12.1 (from versions: none)
ERROR: No matching distribution found for 2.12.1


ERROR: Could not find a version that satisfies the requirement 3.9.4 (from versions: none)
ERROR: No matching distribution found for 3.9.4


ERROR: Could not find a version that satisfies the requirement 0.5.3 (from versions: none)
ERROR: No matching distribution found for 0.5.3


ERROR: Could not find a version that satisfies the requirement 2.12.0 (from versions: none)
ERROR: No matching distribution found for 2.12.0


In [2]:
# USAGE
# python train_mask_detector.py --dataset dataset

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from PIL import Image
import sys
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [3]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 10
BS = 32

In [4]:
# r before string will prevent Python from interpreting escape characters.
DIRECTORY = r"D:\HopeAI\Facemask DL project yogana\dataset"
CATEGORIES = ["with_mask", "without_mask"]

In [5]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

data = []
labels = []

[INFO] loading images...


In [6]:
from imutils import paths

# Define the path to your dataset
dataset_path = r"D:\HopeAI\Facemask DL project yogana\dataset"

# List all image paths in the dataset
imagePaths = list(paths.list_images(dataset_path))

# Print the number of images found and a few example paths
print(f"Found {len(imagePaths)} images.")
print("Example image paths:")
print(imagePaths[:5])  # Print first 5 image paths

Found 3848 images.
Example image paths:
['D:\\HopeAI\\Facemask DL project yogana\\dataset\\without_mask\\0.jpg', 'D:\\HopeAI\\Facemask DL project yogana\\dataset\\without_mask\\0_0_aidai_0014.jpg', 'D:\\HopeAI\\Facemask DL project yogana\\dataset\\without_mask\\0_0_aidai_0029.jpg', 'D:\\HopeAI\\Facemask DL project yogana\\dataset\\without_mask\\0_0_aidai_0043.jpg', 'D:\\HopeAI\\Facemask DL project yogana\\dataset\\without_mask\\0_0_aidai_0074.jpg']


In [16]:
# Initialize data and labels as lists
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

# Convert the data and labels lists to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

print("Data shape:", data.shape)
print("Labels shape:", labels.shape)


Data shape: (3848, 224, 224, 3)
Labels shape: (3848,)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [23]:
data = data.reshape(data.shape[0], -1)
print(data.shape)
#ValueError: Found array with dim 3. None expected <= 2.

(3848, 150528)


In [25]:
print(labels.shape)

(3848, 3, 2)


In [26]:
labels = labels.reshape(labels.shape[0], -1)
print(labels.shape)

(3848, 6)


In [27]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [28]:
# Construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [29]:
# Construct the validation image generator
val_aug = ImageDataGenerator()

In [30]:
# Create data generators
train_generator = aug.flow(trainX, trainY, batch_size=BS)
val_generator = val_aug.flow(testX, testY, batch_size=BS)

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (3078, 150528))

In [31]:
train_generator 

NameError: name 'train_generator' is not defined

In [ ]:
#!pip install h5py

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [ ]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# Compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# Train the head of the network
print("[INFO] training head...")
H = model.fit(
    train_generator,
    steps_per_epoch=len(trainX) // BS,
    validation_data=val_generator,
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

In [ ]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)


In [ ]:
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))


In [ ]:
# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("Face_Mask_Detector.h5")

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")